# ANC Ollama: Wikipedia + FAISS + Ollama (Databricks-Parity)

This notebook mirrors the `Config` and `Wikipedia Data Loader` sections from
`docs/ideation-kernel/01_agentic_wikipedia_aimpoint_interview.ipynb`, but swaps
Databricks components for local Ollama components on macOS.

Provider mapping used here:
- `ChatDatabricks` -> `ChatOllama`
- `DatabricksEmbeddings` -> `OllamaEmbeddings`
- Same `WikipediaLoader` + `FAISS` retrieval flow


## Workflow Diagram

This diagram matches the current notebook flow:

1. Define config (`query_terms`, `max_docs`, `k`, embedding model, chat model).
2. Load Wikipedia documents with `WikipediaLoader`.
3. Embed documents with `OllamaEmbeddings`.
4. Build FAISS index and persist it to disk (`~/DATA/naturalist-companion/faiss/anc_ollama` by default).
5. Run similarity search for the example question.
6. Generate an answer with `ChatOllama`.

Run the next code cell to render the diagram inline.

Execution is split into `Stage 1/3`, `Stage 2/3`, and `Stage 3/3` with timing/status prints.


In [ ]:
from IPython.display import Markdown, SVG, display
import requests

PLANTUML_SOURCE = """
@startuml
title ANC Ollama Notebook Flow
start
:Define config;
:Load Wikipedia docs;
:Embed docs with OllamaEmbeddings;
:Build FAISS index;
:Persist FAISS index to ~/DATA or ANC_FAISS_DIR;
:Run similarity search;
:Answer with ChatOllama;
stop
@enduml
"""


def render_plantuml(source: str) -> None:
    errors = []

    # Preferred path: Python PlantUML client (if installed).
    try:
        from plantuml import PlantUML  # type: ignore

        client = PlantUML(url="http://www.plantuml.com/plantuml/svg/")
        svg_url = client.get_url(source)
        resp = requests.get(svg_url, timeout=20)
        resp.raise_for_status()
        display(SVG(resp.text))
        return
    except Exception as exc:
        errors.append(f"plantuml library renderer failed: {exc}")

    # Fallback path: Kroki PlantUML HTTP endpoint.
    try:
        resp = requests.post(
            "https://kroki.io/plantuml/svg",
            data=source.encode("utf-8"),
            headers={"Content-Type": "text/plain"},
            timeout=20,
        )
        resp.raise_for_status()
        display(SVG(resp.text))
        return
    except Exception as exc:
        errors.append(f"kroki renderer failed: {exc}")

    display(Markdown("PlantUML render failed. Showing source and errors."))
    for item in errors:
        print(item)
    print('\nPlantUML source:\n')
    print(source)


render_plantuml(PLANTUML_SOURCE)


In [ ]:
# Uncomment in fresh environments:
# %pip install -q -r ../requirements-ollama-dev.txt
# %pip install -q plantuml
# %pip install -q ipywidgets


In [ ]:
#######################################################################################################
###### Python Package Imports for this notebook                                                  ######
#######################################################################################################

# LangChain moved WikipediaLoader in newer releases; keep backward compatibility.
try:
    from langchain_community.document_loaders import WikipediaLoader
except ImportError:
    from langchain.document_loaders import WikipediaLoader

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama, OllamaEmbeddings


#######################################################################################################
###### Config (Define LLMs, Embeddings, Vector Store, Data Loader specs)                         ######
#######################################################################################################

# DataLoader Config
query_terms = [
    "roadcut",
    "geology",
    "sedimentary rock",
    "stratigraphy",
]
max_docs = 3  # Fast local iteration setting.

# Retriever Config
k = 1
EMBEDDING_MODEL = "nomic-embed-text"
OLLAMA_BASE_URL = "http://localhost:11434"

# LLM Config
LLM_MODEL = "llama3.2:3b"
TEMPERATURE = 0.0

example_question = "What is a roadcut in geology?"


In [ ]:
#######################################################################################################
###### Stage 1/3: Wikipedia Data Load                                                            ######
#######################################################################################################

from time import perf_counter

print("[stage 1/3] Starting Wikipedia document load...")
query = " ".join(query_terms) if isinstance(query_terms, list) else query_terms
print(f"[stage 1/3] query={query!r}, max_docs={max_docs}")

t0 = perf_counter()
docs = WikipediaLoader(query=query, load_max_docs=max_docs).load()
t1 = perf_counter()

print(f"[stage 1/3] Loaded {len(docs)} document(s) in {t1 - t0:.2f}s")
if not docs:
    raise RuntimeError("No documents loaded from Wikipedia. Adjust query_terms/max_docs and re-run stage 1.")

print("[stage 1/3] Sample titles:")
for i, doc in enumerate(docs[:3], start=1):
    title = str((doc.metadata or {}).get("title") or f"doc_{i}")
    source = str((doc.metadata or {}).get("source") or "n/a")
    print(f"  {i}. {title} ({source})")


In [ ]:
#######################################################################################################
###### Stage 2/3: Build + Save FAISS Index, Then Retrieve                                        ######
#######################################################################################################

import os
from pathlib import Path
from time import perf_counter

if "docs" not in globals() or not docs:
    raise RuntimeError("`docs` not found. Run Stage 1/3 first.")

print(f"[stage 2/3] Building embeddings with model={EMBEDDING_MODEL!r} at {OLLAMA_BASE_URL!r}...")
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL, base_url=OLLAMA_BASE_URL)

t2 = perf_counter()
vector_store = FAISS.from_documents(docs, embeddings)
t3 = perf_counter()
print(f"[stage 2/3] Built FAISS index in {t3 - t2:.2f}s")

faiss_base = os.environ.get("ANC_FAISS_DIR", "").strip()
if faiss_base:
    faiss_dir = (Path(faiss_base).expanduser() / "anc_ollama").resolve()
else:
    faiss_dir = (Path.home() / "DATA" / "naturalist-companion" / "faiss" / "anc_ollama").resolve()

faiss_dir.mkdir(parents=True, exist_ok=True)
vector_store.save_local(str(faiss_dir))
print(f"[stage 2/3] Saved FAISS index to: {faiss_dir}")

print(f"[stage 2/3] Running similarity search for question={example_question!r}, k={k}...")
results = vector_store.similarity_search(example_question, k=k)
print(f"[stage 2/3] Retrieved {len(results)} result(s)")

for i, res in enumerate(results, start=1):
    title = str((res.metadata or {}).get("title") or f"result_{i}")
    source = str((res.metadata or {}).get("source") or "n/a")
    snippet = str(res.page_content or "")[:220].replace("\n", " ")
    print(f"  {i}. {title} ({source})")
    print(f"     {snippet}...")


In [ ]:
#######################################################################################################
###### Stage 3/3: Generate Answer with ChatOllama                                                ######
#######################################################################################################

from time import perf_counter

if "results" not in globals():
    raise RuntimeError("`results` not found. Run Stage 2/3 first.")

print(f"[stage 3/3] Generating answer with model={LLM_MODEL!r} at {OLLAMA_BASE_URL!r}...")
llm = ChatOllama(model=LLM_MODEL, base_url=OLLAMA_BASE_URL, temperature=TEMPERATURE)

prompt = f"Answer concisely using Wikipedia-grounded context. Question: {example_question}"
t4 = perf_counter()
response = llm.invoke(prompt)
t5 = perf_counter()

print(f"[stage 3/3] LLM response received in {t5 - t4:.2f}s")
print("\nAnswer:\n")
print(response.content)


## Local Ollama Prerequisites (macOS)

- Ensure Ollama is installed and running: `ollama serve`
- Pull the embedding model: `ollama pull nomic-embed-text`
- Pull the chat model used in this notebook: `ollama pull llama3.2:3b`
- Keep `notebooks/anc_dbrx.ipynb` for Databricks-specific libraries and endpoints
- Keep `notebooks/anc_gcp.ipynb` for Vertex AI-specific libraries and endpoints

- If you hit macOS OpenMP kernel crashes, launch Jupyter with `KMP_DUPLICATE_LIB_OK=TRUE`.
- Optional PlantUML Python library: `pip install plantuml`

- Optional higher-quality model for final checks: `ollama pull llama3.1:8b`
- If you see `TqdmWarning: IProgress not found`, install widgets in this env: `pip install ipywidgets`, then restart kernel.
- The tqdm warning is non-fatal; notebook execution still works without widget progress bars.
